## Imports

In [38]:
!pip install dash

In [39]:
!pip install jupyter-dash

In [40]:
!pip install dash_mantine_components

In [41]:
!pip install mtranslate==1.8

In [42]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import plotly as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import dash
from dash import dcc
from dash import callback
from dash import Dash, html
from dash.dependencies import Input, Output
import dash_mantine_components as dmc

## Countries and Continents Divisions

In [43]:
options = []

#Europe
european_union = ['European Union']
eastern_europe = ['Romania','Poland','Greece','Bulgaria','Hungary']
western_europe = ['Norway','France','Germany','Spain','Sweden','Italy']

for country in european_union:
    options.append({"value": country, "label": country, "group": "Europan Union"})

for country in eastern_europe:
    options.append({"value": country, "label": country, "group": "Eastern Europe"})

for country in western_europe:
    options.append({"value": country, "label": country, "group": "Western Europe"})

## Covid Data Recap

In [44]:
covid_df = pd.read_csv('global/dataglobal/clean_covid_recap.csv')
covid_df.drop(columns=['Unnamed: 0'],inplace=True)
covid_df.rename(columns={'date':'TIME_PERIOD'},inplace=True)
covid_df

,TIME_PERIOD,Countries,Deaths,Cases,stringency_index
0,2020-01-05,World,NaN,NaN,NaN
1,2020-01-06,World,NaN,NaN,NaN
2,2020-01-07,World,NaN,NaN,NaN
3,2020-01-08,World,NaN,NaN,NaN
4,2020-01-09,World,NaN,NaN,NaN
...,...,...,...,...,...
375435,2024-02-28,Zimbabwe,0.123,0.306,NaN
375436,2024-02-29,Zimbabwe,0.123,0.306,NaN
375437,2024-03-01,Zimbabwe,0.123,0.306,NaN
375438,2024-03-02,Zimbabwe,0.123,0.306,NaN


## Home Page With Interactive Globe

In [45]:
import plotly.graph_objects as go
import pandas as pd

df = pd.read_csv('global/dataglobal/clean_countrycodes.csv')
df.rename(columns={'Unnamed: 0':'COUNTRY','Alpha3':'CODE'},inplace=True)
df['Cases'] = np.log(df['Cases'])


globe = go.Figure(go.Choropleth(
    z=df['Cases'],  # Valores para o mapa de calor
    colorscale='Reds',  # Esquema de cores
    text=df['COUNTRY'],  # Texto que aparecerá quando passar o mouse sobre os países
    locations=df['CODE'],  # Códigos dos países
    zmin=df['Cases'].min(),  # Valor mínimo para o mapa de calor
    zmax=df['Cases'].max(),  # Valor máximo para o mapa de calor
    colorbar_title='Avg Weekly Cases (per Million)',  # Título da barra de cores
    marker_line_width=1  # Largura da linha do marcador
))

# Configurações de layout
globe.update_layout(
    geo=dict(
        showframe = False,
        showcoastlines = False,
        projection = dict(type = 'orthographic'),
        resolution = 110,
        showcountries = True,
        countrycolor = '#d1d1d1',
        showocean = True,
        oceancolor = '#c9d2e0',
        showlakes = True,
        lakecolor = '#99c0db',
        showrivers = True,
        rivercolor = '#99c0db'
    ),
    width=1200,  # Largura da figura
    height=900  # Altura da figura
)

# Exibindo o gráfico
globe.show()

c:\Users\manue\anaconda3\envs\MachineLearning\lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning:

divide by zero encountered in log



In [46]:
#Defining layout for the home page
def homepage_layout():
    return html.Div( #The Header
        children =[
            html.Div(
                className="app-header",
                children=[
                    html.H1(children="Covid Worldwide Analysis", className='app-header--title'),
                    html.P(children="Evolution of the economy before, during and after the Covid pandemic. Continental and global analysis.",
                            className='app-header--description'),
                ]
            ),
            html.Div( #The Menu
                className="app-nav",
                children=[
                    html.Ul(
                        children=[
                            html.A('Employee Compensation', href='/employee_compensation'),
                            html.A('Employment Rate', href='/employment_rate'),
                            html.A('Final Consumption Expenditure', href='/final_consumption_expenditure'),
                            html.A('GFCF', href='/gfcf'),
                            html.A('GDP', href='/gdp')
                        ]
                    )
                ]
            ),
            html.Div( #The Globe
                children=[
                    dcc.Graph(figure=globe),       
                ],
                style = {"display": "flex",
                    "flex-direction": "row",
                    "justify-content": "center"
                }
            )
        ]
    )

## Page 1: Employee compensation

In [47]:
emp_comp = pd.read_csv("global/dataglobal/clean_employee_compensation.csv")
emp_comp = emp_comp[emp_comp["Zone de référence"].isin(['France', 'Poland', 'Sweden','European Union (27 countries as of 02/01/2020)', 'Romania', 'Hungary', 'Italy', 'Norway', 'Bulgaria', 'Spain', 'Germany', 'Greece'])]
emp_comp = emp_comp[emp_comp['Transaction'].isin(['Rémunération des salariés'])]
emp_comp = emp_comp[emp_comp['Activité économique'].isin(['Construction','Total - Toutes les activités','Industrie (sauf construction)'])]
emp_comp["Zone de référence"] = np.where(emp_comp["Zone de référence"] == 'European Union (27 countries as of 02/01/2020)','European Union',emp_comp["Zone de référence"])
emp_comp.rename(columns={'Zone de référence':'Countries'},inplace=True)
emp_comp.drop(columns=['Unnamed: 0'],inplace=True)
emp_comp.dropna(axis=0,inplace=True)
emp_comp.head()

,STRUCTURE_NAME,Countries,Secteur institutionnel,Transaction,Activité économique,TIME_PERIOD,OBS_VALUE,Multiplicateur d'unité
459,Rémunération trimestrielle des salariés par ac...,Romania,Economie totale,Rémunération des salariés,Construction,1995-06-01,0.163842,Millions
460,Rémunération trimestrielle des salariés par ac...,Romania,Economie totale,Rémunération des salariés,Construction,1995-09-01,0.152913,Millions
461,Rémunération trimestrielle des salariés par ac...,Romania,Economie totale,Rémunération des salariés,Construction,1995-12-01,0.149474,Millions
462,Rémunération trimestrielle des salariés par ac...,Romania,Economie totale,Rémunération des salariés,Construction,1996-03-01,0.179487,Millions
463,Rémunération trimestrielle des salariés par ac...,Romania,Economie totale,Rémunération des salariés,Construction,1996-06-01,0.156832,Millions


In [48]:
merged_emp_comp = pd.merge(emp_comp, covid_df, on=['Countries', 'TIME_PERIOD'], how='left')

merged_emp_comp['Cases'] = (merged_emp_comp['Cases']*merged_emp_comp['OBS_VALUE'].mean())/merged_emp_comp['Cases'].mean()

In [49]:
#Define layout for page 1 (Employee Compensation)
def page1_layout():
    return html.Div(
        html.Div(
            className='wrapper',
            children=[
                html.Div( #The Header
                    className="app-header",
                    children=[
                        html.H1(children="Employee Compensation", className='app-header--title'),
                        html.P(
                            children="Quarterly remuneration of European employees by economic activity.",
                            className='app-header--description'),
                    ],
                ),
                html.Div( #The Home Page link
                    children=[
                        dcc.Link('Home Page', href='/'),
                    ],
                    style={'width': '100%', 'margin-top':'-120px',}
                ),
                html.Div( #The dropdown list and the graphic
                    children=[
                        dmc.MultiSelect( #Dropdown list
                            label="Select the Countries you want to analyse",
                            placeholder="Select one or more",
                            value=["European Union"],
                            data=options,
                            id='countries_filter',
                            style={'width': '100%', 'margin-top':'110px',},
                        ),
                        html.Div( #Graphic (Code is in the dash part)
                            className = 'plot_card',
                            children=[
                                dcc.Graph(id='emp_comp_plot'),        
                            ],
                        )
                    ],
                )
            ]
        )
    )

## Page 2: Employment Rate

In [50]:
emp_rate = pd.read_csv("global/dataglobal/clean_employment.csv")
emp_rate = emp_rate[emp_rate["Zone de référence"].isin(['France', 'Poland', 'Sweden', 'European Union (27 countries from 02/01/2020)', 'Romania', 'Hungary', 'Italy', 'Norway', 'Bulgaria', 'Spain', 'Germany', 'Greece'])]
emp_rate = emp_rate[emp_rate['Activité économique'].isin(['Construction','Total - Toutes les activités','Industrie (sauf construction)','Information et communication'])]
emp_rate["Zone de référence"] = np.where(emp_rate["Zone de référence"] == 'European Union (27 countries as of 02/01/2020)','European Union',emp_rate["Zone de référence"])
emp_rate.rename(columns={'Zone de référence':'Countries'},inplace=True)
emp_rate.drop(columns=['Unnamed: 0'],inplace=True)
emp_rate.dropna(axis=0,inplace=True)
emp_rate.head()

,Fréquence d'observation,Countries,Transaction,Activité économique,Unité de mesure,TIME_PERIOD,OBS_VALUE,Multiplicateur d'unité
1803,Trimestrielle,Italy,Emploi total,Information et communication,Heures,1995-06-01,0.000992,Millions
1804,Trimestrielle,Italy,Emploi total,Information et communication,Heures,1995-09-01,0.007929,Millions
1805,Trimestrielle,Italy,Emploi total,Information et communication,Heures,1995-12-01,0.020649,Millions
1806,Trimestrielle,Italy,Emploi total,Information et communication,Heures,1996-03-01,0.004335,Millions
1807,Trimestrielle,Italy,Emploi total,Information et communication,Heures,1996-06-01,0.004317,Millions


In [51]:
merged_emp_rate = pd.merge(emp_rate, covid_df, on=['Countries', 'TIME_PERIOD'], how='left')

In [52]:
#Define layout for page 2 (Employment Rate)
def page2_layout():
    return html.Div(
        html.Div(
            className='wrapper',
            children=[
                html.Div( #The Header
                    className="app-header",
                    children=[
                        html.H1(children="Employment Rate", className='app-header--title'),
                        html.P(
                            children="Employment rate variation measured in hours",
                            className='app-header--description'),
                    ],
                ),
                html.Div( #The Home Page link
                    children=[
                        dcc.Link('Home Page', href='/'),
                    ],
                    style={'width': '100%', 'margin-top':'-120px',}
                ),
                html.Div( #The dropdown list and the graphic
                    children=[
                        dmc.MultiSelect( #Dropdown list
                            label="Select the Countries you want to analyse",
                            placeholder="Select one or more",
                            value=["France"],
                            data=options,
                            id='countries_filter',
                            style={'width': '100%', 'margin-top':'110px',},
                        ),
                        html.Div( #Graphic (Code is in the dash part)
                            className = 'plot_card',
                            children=[
                                dcc.Graph(id='emp_rate_plot'),       
                            ],
                        )
                    ],
                )
            ]
        )
    )

## Page 3: Final Consumption Expenditure

In [53]:
consumption = pd.read_csv("global/dataglobal/clean_finalconsumptionexpenditure.csv")
consumption = consumption[consumption["Zone de référence"].isin(['France', 'Colombia', 'Poland', 'New Zealand', 'United States', 'Sweden', 'European Union (27 countries as of 01/02/2020)', 'Romania', 'Hungary', 'Italy', 'Norway', 'Bulgaria', 'Spain', 'Japan', 'Australia', 'Korea', 'Canada', 'Germany', 'Chile', 'Greece'])]
consumption["Zone de référence"] = np.where(consumption["Zone de référence"] == 'European Union (27 countries as of 02/01/2020)','European Union',consumption["Zone de référence"])
consumption.rename(columns={'Zone de référence':'Countries'},inplace=True)
consumption.drop(columns=['Unnamed: 0'],inplace=True)
consumption.dropna(axis=0,inplace=True)
consumption.head()

,Countries,Secteur institutionnel,TIME_PERIOD,OBS_VALUE,Multiplicateur d'unité,CURRENCY
137,Norway,Ménages,1978-06-01,-0.000040,Millions,NOK
138,Norway,Ménages,1978-09-01,0.019939,Millions,NOK
139,Norway,Ménages,1978-12-01,-0.017099,Millions,NOK
140,Norway,Ménages,1979-03-01,0.027248,Millions,NOK
141,Norway,Ménages,1979-06-01,0.021286,Millions,NOK


In [54]:
merged_consumption = pd.merge(consumption, covid_df, on=['Countries', 'TIME_PERIOD'], how='left')

In [55]:
#Define layout for page 3 (Employment Rate)
def page3_layout():
    return html.Div(
        html.Div(
            className='wrapper',
            children=[
                html.Div( #The Header
                    className="app-header",
                    children=[
                        html.H1(children="Final Consumption Expenditure", className='app-header--title'),
                        html.P(
                            children="Private final consumption expenditure per continent",
                            className='app-header--description'),
                    ],
                ),
                html.Div( #The Home Page link
                    children=[
                        dcc.Link('Home Page', href='/'),
                    ],
                    style={'width': '100%', 'margin-top':'-120px',}
                ),
                html.Div( #The dropdown list and the graphic
                    children=[
                        dmc.MultiSelect( #Dropdown list
                            label="Select the Countries you want to analyse",
                            placeholder="Select one or more",
                            value=["France"],
                            data=options,
                            id='countries_filter',
                            style={'width': '100%', 'margin-top':'110px',},
                        ),
                        html.Div( #Graphic (Code is in the dash part)
                            className = 'plot_card',
                            children=[
                                dcc.Graph(id='consumption_plot'),       
                            ],
                        )
                    ],
                )
            ]
        )
    )

## Page 4: GFCF

In [56]:
gfcf = pd.read_csv("global/dataglobal/clean_GFCF.csv")
gfcf = gfcf[gfcf["Zone de référence"].isin(['France', 'Colombia', 'Poland', 'New Zealand', 'United States', 'Sweden', 'European Union (27 countries as of 01/02/2020)', 'Romania', 'Hungary', 'Italy', 'Norway', 'Bulgaria', 'Spain', 'Japan', 'Australia', 'Korea', 'Canada', 'Germany', 'Chile', 'Greece'])]
gfcf["Zone de référence"] = np.where(gfcf["Zone de référence"] == 'European Union (27 countries as of 02/01/2020)','European Union',gfcf["Zone de référence"])
gfcf.rename(columns={'Zone de référence':'Countries'},inplace=True)
gfcf.drop(columns=['Unnamed: 0'],inplace=True)
gfcf.dropna(axis=0,inplace=True)
gfcf.head()

,Countries,Secteur institutionnel,Instruments financiers et actifs non financiers,TIME_PERIOD,OBS_VALUE,Multiplicateur d'unité,CURRENCY
120,France,Economie totale,"Actifs fixes, brut",1981-03-01,-0.015508,Millions,EUR
121,France,Economie totale,"Actifs fixes, brut",1981-06-01,-0.011916,Millions,EUR
122,France,Economie totale,"Actifs fixes, brut",1981-09-01,-0.011932,Millions,EUR
123,France,Economie totale,"Actifs fixes, brut",1981-12-01,0.001023,Millions,EUR
124,France,Economie totale,"Actifs fixes, brut",1982-03-01,-0.000121,Millions,EUR


In [57]:
merged_gfcf = pd.merge(gfcf, covid_df, on=['Countries', 'TIME_PERIOD'], how='left')

In [58]:
# Define layout for page 4 (GFCF)
def page4_layout():
    return html.Div(
        html.Div(
            className='wrapper',
            children=[
                html.Div( #The Header
                    className="app-header",
                    children=[
                        html.H1(children="Gross Fixed Capital Formation", className='app-header--title'),
                        html.P(
                            children="... write something here",
                            className='app-header--description'),
                    ],
                ),
                html.Div( #The Home Page link
                    children=[
                        dcc.Link('Home Page', href='/'),
                    ],
                    style={'width': '100%', 'margin-top':'-120px',}
                ),
                html.Div( #The dropdown list and the graphic
                    children=[
                        dmc.MultiSelect( #Dropdown list
                            label="Select the Countries you want to analyse",
                            placeholder="Select one or more",
                            value=["France"],
                            data=options,
                            id='countries_filter',
                            style={'width': '100%', 'margin-top':'110px',},
                        ),
                        html.Div( #Graphic (Code is in the dash part)
                            className = 'plot_card',
                            children=[
                                dcc.Graph(id='gfcf_plot'),       
                            ],
                        )
                    ],
                )
            ],
        )
    )

## Page 5: GDP

In [59]:
gdp = pd.read_csv("global/dataglobal/clean_GFCF.csv")
gdp = gdp[gdp["Zone de référence"].isin(['France', 'Colombia', 'Poland', 'New Zealand', 'United States', 'Sweden', 'European Union (27 countries as of 01/02/2020)', 'Romania', 'Hungary', 'Italy', 'Norway', 'Bulgaria', 'Spain', 'Japan', 'Australia', 'Korea', 'Canada', 'Germany', 'Chile', 'Greece'])]
gdp["Zone de référence"] = np.where(gdp["Zone de référence"] == 'European Union (27 countries as of 02/01/2020)','European Union',gdp["Zone de référence"])
gdp.rename(columns={'Zone de référence':'Countries'},inplace=True)
gdp.drop(columns=['Unnamed: 0'],inplace=True)
gdp.dropna(axis=0,inplace=True)
gdp.head()

,Countries,Secteur institutionnel,Instruments financiers et actifs non financiers,TIME_PERIOD,OBS_VALUE,Multiplicateur d'unité,CURRENCY
120,France,Economie totale,"Actifs fixes, brut",1981-03-01,-0.015508,Millions,EUR
121,France,Economie totale,"Actifs fixes, brut",1981-06-01,-0.011916,Millions,EUR
122,France,Economie totale,"Actifs fixes, brut",1981-09-01,-0.011932,Millions,EUR
123,France,Economie totale,"Actifs fixes, brut",1981-12-01,0.001023,Millions,EUR
124,France,Economie totale,"Actifs fixes, brut",1982-03-01,-0.000121,Millions,EUR


In [60]:
merged_gdp = pd.merge(gdp, covid_df, on=['Countries', 'TIME_PERIOD'], how='left')

In [61]:
#Define layout for page 5 (Employment Rate)
def page5_layout():
    return html.Div(
        html.Div(
            className='wrapper',
            children=[
                html.Div( #The Header
                    className="app-header",
                    children=[
                        html.H1(children="GDP", className='app-header--title'),
                        html.P(
                            children="... write something here",
                            className='app-header--description'),
                    ],
                ),
                html.Div( #The Home Page link
                    children=[
                        dcc.Link('Home Page', href='/'),
                    ],
                    style={'width': '100%', 'margin-top':'-120px',}
                ),
                html.Div( #The dropdown list and the graphic
                    children=[
                        dmc.MultiSelect( #Dropdown list
                            label="Select the Countries you want to analyse",
                            placeholder="Select one or more",
                            value=["European Union"],
                            data=options,
                            id='countries_filter',
                            style={'width': '100%', 'margin-top':'110px',},
                        ),
                        html.Div( #Graphic (Code is in the dash part)
                            className = 'plot_card',
                            children=[
                                dcc.Graph(id='gdp_plot'),       
                            ],
                        )
                    ],
                )
            ]
        )
    )

# Main

In [62]:
'''import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import dash_core_components as dcc
import dash_html_components as html
from dash import Dash
from dash import callback_context
from dash.exceptions import PreventUpdate
from dash.dependencies import Input, Output, State'''

# Initialize the Dash app
app = dash.Dash(__name__, suppress_callback_exceptions=True, index_string='''<!DOCTYPE html>
<html>
    <head>
        <link rel="preconnect" href="https://fonts.googleapis.com">
        <link rel="preconnect" href="https://fonts.gstatic.com" crossorigin>
        <link href="https://fonts.googleapis.com/css2?family=Angkor&display=swap" rel="stylesheet">
        {%metas%}
        <title>{%title%}</title>
        {%favicon%}
        {%css%}
    </head>
    <body>
        {%app_entry%}
        <footer>
            {%config%}
            {%scripts%}
            {%renderer%}
        </footer>
    </body>
</html>
''')

#Upgrade page 1 Graphic (Employee Compensation)
@app.callback(
    Output('emp_comp_plot', 'figure'),
    Input('countries_filter', 'value'))
def update_graph_emp_comp(selected_categories):

    filtered_emp_comp = emp_comp[emp_comp['Countries'].isin(selected_categories)]

    fig = px.line(filtered_emp_comp, x='TIME_PERIOD', y="OBS_VALUE",color='Countries', facet_col="Activité économique", facet_col_wrap=3)

    fig.update_xaxes(rangeslider_visible=True,col=2)
    
    fig.update_layout(xaxis_range=['2000-01-01','2024-04-01'],
                    yaxis_range=[filtered_emp_comp[(filtered_emp_comp['TIME_PERIOD']>='2000-01-01')]['OBS_VALUE'].min()-0.05,filtered_emp_comp[(filtered_emp_comp['TIME_PERIOD']>='2000-01-01')]['OBS_VALUE'].max()+0.05],
                    title_text= "Employee Compensation per European Contry")

    return fig

#Upgrade page 2 Graphic (Employment rate)
@app.callback(
    Output('emp_rate_plot', 'figure'),
    Input('countries_filter', 'value'))
def update_graph_emp_rate(selected_categories):

    filtered_emp_rate = emp_rate[emp_rate['Countries'].isin(selected_categories)]

    fig = px.line(filtered_emp_rate, x='TIME_PERIOD', y="OBS_VALUE",color='Countries',
                  facet_col="Activité économique", facet_col_wrap=2,
                   facet_row_spacing=0.12, height=600)

    fig.update_xaxes(rangeslider_visible=True,row=1)
    
    fig.update_layout(xaxis_range=['2000-01-01','2024-04-01'],
                    yaxis_range=[filtered_emp_rate[(filtered_emp_rate['TIME_PERIOD']>='2000-01-01')]['OBS_VALUE'].min()-0.05,filtered_emp_rate[(filtered_emp_rate['TIME_PERIOD']>='2000-01-01')]['OBS_VALUE'].max()+0.05],
                    title_text= "Employee Rate per European Country")

    return fig

#Upgrade page 3 Graphic (Final Consumption Expenditure)
@app.callback(
    Output('consumption_plot', 'figure'),
    Input('countries_filter', 'value'))
def update_graph_consumption(selected_categories):

    filtered_consumption = consumption[consumption['Countries'].isin(selected_categories)]

    fig = px.line(filtered_consumption, x='TIME_PERIOD', y="OBS_VALUE",color='Countries')

    fig.update_xaxes(rangeslider_visible=True,row=1)
    
    fig.update_layout(xaxis_range=['2000-01-01','2024-04-01'],
                    yaxis_range=[filtered_consumption[(filtered_consumption['TIME_PERIOD']>='2000-01-01')]['OBS_VALUE'].min()-0.05,filtered_consumption[(filtered_consumption['TIME_PERIOD']>='2000-01-01')]['OBS_VALUE'].max()+0.05],
                    title_text= "Final Consumption Expenditure")

    return fig

#Upgrade page 4 Graphic (GFCF)
@app.callback(
    Output('gfcf_plot', 'figure'),
    Input('countries_filter', 'value'))
def update_graph(selected_categories):

    filtered_gfcf = gfcf[gfcf['Countries'].isin(selected_categories)]

    fig = px.line(filtered_gfcf, x='TIME_PERIOD', y="OBS_VALUE",color='Countries',facet_col="Instruments financiers et actifs non financiers", facet_col_wrap=3)

    fig.update_xaxes(rangeslider_visible=True,row=1)

    fig.update_layout(xaxis_range=['2000-01-01','2024-04-01'],
                    yaxis_range=[filtered_gfcf[(filtered_gfcf['TIME_PERIOD']>='2000-01-01')]['OBS_VALUE'].min()-0.05,filtered_gfcf[(filtered_gfcf['TIME_PERIOD']>='2000-01-01')]['OBS_VALUE'].max()+0.05],
                    title_text= "Gross Fixed Capital Formation")

    return fig

#Upgrade page 5 Graphic (GDP)
@app.callback(
    Output('gdp_plot', 'figure'),
    Input('countries_filter', 'value'))
def update_graph_consumption(selected_categories):

    filtered_gdp = gdp[gdp['Countries'].isin(selected_categories)]

    fig = px.line(filtered_gdp, x='TIME_PERIOD', y="OBS_VALUE",color='Countries', facet_col='Instruments financiers et actifs non financiers',facet_col_wrap=3)

    fig.update_xaxes(rangeslider_visible=True,row=1)
    
    fig.update_layout(xaxis_range=['2000-01-01','2024-04-01'],
                    yaxis_range=[filtered_gdp[(filtered_gdp['TIME_PERIOD']>='2000-01-01')]['OBS_VALUE'].min()-0.05,filtered_gdp[(filtered_gdp['TIME_PERIOD']>='2000-01-01')]['OBS_VALUE'].max()+0.05],
                    title_text= "Employee Rate per European Country")

    return fig

# Define callback to display the corresponding layout based on the current pathname
@app.callback(Output('page-content', 'children'),
              [Input('url', 'pathname')])
def display_page(pathname):
    if pathname == '/employee_compensation':
        return page1_layout()
    elif pathname == '/employment_rate':
        return page2_layout()
    elif pathname == '/final_consumption_expenditure':
        return page3_layout()
    elif pathname == '/gfcf':
        return page4_layout()
    elif pathname == '/gdp':
        return page5_layout()
    else:
        return homepage_layout()

# Define the layout of the entire app
app.layout = html.Div([
    dcc.Location(id='url', refresh=False),
    html.Div(id='page-content')
])

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)

c:\Users\manue\anaconda3\envs\MachineLearning\lib\site-packages\plotly\express\_core.py:2065: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.

c:\Users\manue\anaconda3\envs\MachineLearning\lib\site-packages\plotly\express\_core.py:2065: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.

c:\Users\manue\anaconda3\envs\MachineLearning\lib\site-packages\plotly\express\_core.py:2065: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.

c:\Users\manue\anaconda3\envs\MachineLearning\lib\site-packages\plotly\express\_core.py:2065: FutureWarning:

When groupi